In [188]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
import binarytree

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs
from utils import averageModels, normalModels

In [189]:
class Arguments():
    def __init__(self):
        self.images = 50000
        self.clients = 20
        self.rounds = 10
        self.epochs = 5
        self.local_batches = 64
        self.lr = 0.01
        self.C = 1.0
        self.drop_rate = 0.0
        self.mu = 0.5
        self.momentum = 0.6
        self.dampening = 1.0
        self.torch_seed = 0
        self.log_interval = 100
        self.iid = 'iid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = False
        self.save_model = False

args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [190]:
writer = open('prox.csv', 'a')

In [191]:
hook = sy.TorchHook(torch)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

In [192]:
# # Download MNIST manually using 'wget' then uncompress the file
# !wget www.di.ens.fr/~lelarge/MNIST.tar.gz
# !tar -zxvf MNIST.tar.gz

In [193]:
global_train, global_test, train_group, test_group = load_dataset(args.clients, args.iid)

Files already downloaded and verified
Files already downloaded and verified


In [194]:
for inx, client in enumerate(clients):
    trainset_ind_list = list(train_group[inx])
    client['trainset'] = getActualImgs(global_train, trainset_ind_list, args.local_batches)
    client['testset'] = getActualImgs(global_test, list(test_group[inx]), args.local_batches)
#     client['samples'] = len(trainset_ind_list) / args.images
    client['samples'] = len(trainset_ind_list) / (len(trainset_ind_list) * args.clients)

In [195]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
global_test_dataset = datasets.CIFAR10('./', train=False, download=True, transform=transform)
global_test_loader = DataLoader(global_test_dataset, batch_size=args.local_batches, shuffle=True)

Files already downloaded and verified


In [196]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [197]:
def ClientUpdate(args, device, client):
    client['model'].train()
#     client['model'].send(client['hook'])
    
    for epoch in range(1, args.epochs + 1):
        for batch_idx, (data, target) in enumerate(client['trainset']):
#             data = data.send(client['hook'])
#             target = target.send(client['hook'])
            
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
            output = client['model'](data)
            loss = F.nll_loss(output, target)
            loss.backward()
            client['optim'].step()
#             client['optim'].step(global_model)
    
            
            if batch_idx % args.log_interval == 0:
#                 loss = loss.get() 
                print('Model {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    client['hook'].id,
                    epoch, batch_idx * args.local_batches, len(client['trainset']) * args.local_batches, 
                    100. * batch_idx / len(client['trainset']), loss))
                
#     client['model'].get() 

In [198]:
def test(args, model, device, test_loader, name):
    model.eval()   
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss for {} model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    writer.write('{},'.format(100. * correct / len(test_loader.dataset)))

In [199]:
class FedProxOptim(optim.Optimizer):
    def __init__(self, params, lr=args.lr, mu=args.mu, momentum=args.momentum, dampening=args.dampening):
        defaults = dict(lr=lr, mu=mu, momentum=momentum, dampening=dampening)
        super(FedProxOptim, self).__init__(params, defaults)
    
    def step(self, global_model=None, closure = None):
        loss = None
        if closure is not None:
            loss = closure()
        for group in self.param_groups:
            lr, mu, momentum, dampening = group['lr'], group['mu'], group['momentum'], group['dampening']
            for p in zip(group['params'], list(global_model.parameters())):
                if p[0].grad is None:
                    continue
                d_p = p[0].grad.data # local model grads
                if momentum != 0:
                    param_state = self.state[p[0]]
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = d_p.clone()
                    else:
                        buf = param_state['momentum_buffer']
                        buf.mul_(momentum).add_(1 - dampening, d_p)
                p[0].data.sub_(group['lr'], (d_p + mu * (p[0].data.clone() - p[1].data.clone())))
                
        return loss

In [200]:
def Aggregate(all_models, sample_client):
    all_dicts = {}
    for client in all_models:
        all_dicts[client] = all_models[client].state_dict()
    output_dict = copy.deepcopy(all_dicts[sample_client])
    for key in all_dicts[sample_client]:
        for val in all_dicts:
            if val == sample_client:
                continue
            output_dict[key] += all_dicts[val][key]
#         output_dict[key] /= float(len(all_dicts))
        output_dict[key] /= float(len(all_models))
    output_model = Net().to(device)
    output_model.load_state_dict(output_dict)
    return output_model

In [201]:
def BtreeAvg(root):
    global global_model
    temp_model = Aggregate({'root':root.value[1], 'global':global_model}, 'global')
    
    if (root.left == None and root.right == None): 
        return temp_model
    left = None
    right = None
    if root.left:
        left = BtreeAvg(root.left)
    if root.right:
        right = BtreeAvg(root.right)

    if left and right:
        return Aggregate({'root':root.value[1], 'left':left, 'right':right}, 'left')
    if left:
        return Aggregate({'root':root.value[1], 'left':left}, 'left')
    if right:
        return Aggregate({'root':root.value[1],'right':right}, 'right')
 

In [202]:
def BtreeAvgSecond(root, final):
    temp_model = Aggregate({'root':root.value[1], 'global':final}, 'global')
    
    if (root.left == None and root.right == None): 
        return temp_model
    left = None
    right = None
    if root.left:
        left = BtreeAvgSecond(root.left, temp_model)
    if root.right:
        right = BtreeAvgSecond(root.right, temp_model)

    if left and right:
        return Aggregate({'root':root.value[1], 'left':left, 'right':right}, 'left')
    if left:
        return Aggregate({'root':root.value[1], 'left':left}, 'left')
    if right:
        return Aggregate({'root':root.value[1],'right':right}, 'right')
 

In [203]:
torch.manual_seed(args.torch_seed)
global_model = Net()
# torch.manual_seed(args.torch_seed)
# normal_avg = Net()

for client in clients:
    torch.manual_seed(args.torch_seed)
    client['model'] = Net().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr, momentum=args.momentum)
#     client['optim'] = optim.Adam(client['model'].parameters(), lr=args.lr)
#     client['optim'] = FedProxOptim(client['model'].parameters(), lr=args.lr, mu=args.mu, momentum=args.momentum, dampening=args.dampening)

tree_List = []
for inds, i in enumerate(clients):
    temp = [None] * 3
    temp[0] = i
    temp[1] = clients[inds]['model']
    temp[2] = clients[inds]['hook'].id
    tree_List.append(temp)

root = None
root = binarytree.insertLevelOrder(tree_List, root, 0, len(tree_List))
# root.printTree()

start_time = time.time()

for fed_round in range(args.rounds):
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))
    
    # number of selected clients
    m = int(max(args.C * args.clients, 1))

    # Selected devices
    np.random.seed(fed_round)
    selected_clients_inds = np.random.choice(range(len(clients)), m, replace=False)
    selected_clients = [clients[i] for i in selected_clients_inds]
    
    # Active devices
    np.random.seed(fed_round)
    active_clients_inds = np.random.choice(selected_clients_inds, int((1-args.drop_rate) * m), replace=False)
    active_clients = [clients[i] for i in active_clients_inds]
    print(len(active_clients))
    
    # Training 
    for client in active_clients:
        ClientUpdate(args, device, client)
        
#     global_model = averageModels(global_model, active_clients)
#     test(args, global_model, device, global_test_loader, 'Global')
#     normal_avg = normalModels(normal_avg, active_clients)
#     test(args, normal_avg, device, global_test_loader, 'Global')
    
    
    # Begin the Btree
#     final_model = BtreeAvg(root)
    final_model = BtreeAvgSecond(root, global_model)
#     glo_model = copy.deepcopy(final_model)
    global_model.load_state_dict(final_model.state_dict())
#     test(args, global_model, device, global_test_loader, 'Global')
#     break
    
#     # Testing 
#     for client in active_clients:
#         test(args, client['model'], device, client['testset'], client['hook'].id)
    
#     # Averaging 
#     global_model = averageModels(global_model, active_clients)
    
    # Testing the average model
    test(args, global_model, device, global_test_loader, 'Global')
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())

writer.write('\n')
end_time = time.time() 
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
writer.write('{:0>2}:{:0>2}:{:05.2f}'.format(int(hours),int(minutes),seconds))
writer.write('\n\n')
writer.close()

if (args.save_model):
    torch.save(global_model.state_dict(), "FedAvg.pt")

20
Model client16 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.323728
Model client16 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.300805
Model client16 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.312474
Model client16 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.309654
Model client16 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.299466
Model client2 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.322279
Model client2 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.292088
Model client2 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.297641
Model client2 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.301197
Model client2 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.301133
Model client13 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.307345
Model client13 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.316091
Model client13 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.300844
Model client13 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.300178
Model client13 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.309526
Model client5 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.300318
Model client5 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.305442
M

Model client13 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.254735
Model client18 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.299006
Model client18 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.293644
Model client18 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.292946
Model client18 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.288177
Model client18 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.270805
Model client17 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.309384
Model client17 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.301164
Model client17 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.299456
Model client17 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.299457
Model client17 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.292113
Model client19 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.296079
Model client19 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.295840
Model client19 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.291762
Model client19 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.283416
Model client19 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.289820
Model client4 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.29351

Model client4 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.261462
Model client4 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.067625
Model client9 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.249765
Model client9 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.256218
Model client9 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.236528
Model client9 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.185240
Model client9 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.120499
Model client3 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.262315
Model client3 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.247466
Model client3 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.169548
Model client3 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.091403
Model client3 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.949201
Model client7 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.248651
Model client7 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.258587
Model client7 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.170902
Model client7 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.055225
Model client7 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.093729
Model client2 

Model client1 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.977567
Model client1 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.084899
Model client1 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.939963
Model client1 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.640182
Model client3 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.884346
Model client3 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.896620
Model client3 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.734397
Model client3 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.752297
Model client3 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.806705
Model client14 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.858219
Model client14 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.983229
Model client14 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.846429
Model client14 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.966577
Model client14 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.818569
Model client20 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.913852
Model client20 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.956919
Model client20 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.922564
Model 

Model client13 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.769867
Model client13 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.762006
Model client13 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.818791
Model client13 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.819298
Model client13 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.888267
Model client14 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.920756
Model client14 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.904785
Model client14 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.828014
Model client14 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.951365
Model client14 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.798059
Model client19 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.719733
Model client19 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.794722
Model client19 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.087219
Model client19 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.772811
Model client19 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.801069
Model client3 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.810307
Model client3 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.806446

Model client12 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.366316
Model client13 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.738015
Model client13 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.650191
Model client13 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.727944
Model client13 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.548644
Model client13 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.677815

Test set: Average loss for Global model: 1.6733, Accuracy: 3868/10000 (39%)

20
Model client18 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.596408
Model client18 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.957762
Model client18 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.785295
Model client18 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.074211
Model client18 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.672795
Model client4 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.535476
Model client4 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.983038
Model client4 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.731843
Model client4 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.525067
Model client4 Train Epoch: 5 [0/2560 (

Model client15 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.616530
Model client15 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.641246
Model client15 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.618675
Model client7 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.686452
Model client7 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.745269
Model client7 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.274652
Model client7 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.546034
Model client7 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.404091
Model client20 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.554260
Model client20 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.627853
Model client20 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.750132
Model client20 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.637403
Model client20 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.521886
Model client12 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.721355
Model client12 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.671141
Model client12 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.657077
Model client12 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.225890
Mo

Model client15 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.515455
Model client15 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.521193
Model client15 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.475621
Model client15 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.765137
Model client1 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.586772
Model client1 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.507795
Model client1 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.764699
Model client1 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.578967
Model client1 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.444180
Model client2 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.408780
Model client2 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.833284
Model client2 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.403444
Model client2 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.642575
Model client2 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.332177
Model client18 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.510913
Model client18 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.592302
Model client18 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.827887
Model c

In [204]:
clients[0]['optim']

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.6
    nesterov: False
    weight_decay: 0
)